## Feature Engineering for Machine Learning

### CHAPTER 1. Creating Features

#### 1.1 Why generate features?

### CHAPTER 2. Dealing with messy data

#### 2.1

### CHAPTER 3. Conforming to Statistical Assumptions

#### 3.1

### CHAPTER 4. Dealing with Text Data

#### 4.1